In [105]:
import tkinter as tk
import os
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
# import des bibliothèques requises
from roboflow import Roboflow
from PIL import Image, ImageDraw

# configuration du client Roboflow-----------------
rf = Roboflow(api_key="08aLPIwn6wT5XhvIOvIx")
# récupération du projet et du modèle depuis Roboflow
project = rf.workspace().project("coins-r6on6")
model = project.version(2).model

#creer fenetre 
window = tk.Tk()
window.title("COINS Detector")
window.geometry("1080x740")#taille ecran
window.minsize(550,550)#taille Minimal par défaut
window.iconbitmap("icone/icone.ico") #Icon
window.config(background='#14213D')#Background hexadecimal
photo = None
img_path = None


loading Roboflow workspace...
loading Roboflow project...


In [106]:
     # definir une methode pour lire une image a partir d'une repertoire et l'afficher dans le label.
def getImg():
    sum = 0
    # lire une image a partir d'une repertoire
    global img_path 
    img_path = filedialog.askopenfilename(title='choisir un fichier')
    img = Image.open(img_path)
    width, height = img.size
    img =  img.resize((500, int(500/width*height)))
    
    if(img_path!=None):
            #-----------------------------------------
            predictions = model.predict(img_path, confidence=40, overlap=30).json()
            model.predict(img_path, confidence=40, overlap=30).save("prediction.jpg")
            
            imgOriginal = cv2.imread(img_path) 
            imgGris =  cv2.cvtColor(imgOriginal,  cv2.COLOR_BGR2GRAY)
            seuil,imgbw = cv2.threshold(imgGris,0,255, cv2.THRESH_OTSU)
            contours, hierarchy = cv2.findContours(imgbw,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(imgOriginal, contours, -1, (0,0,255), 2)
            filename = "imgageContours.jpg"
            
                # afficher l'image Original dans le label  
            photo = ImageTk.PhotoImage(img.resize((450, 450)))  
            labelImg1.configure(image = photo)
            labelImg1.image = photo

            if os.path.exists(filename):
                os.remove(filename)

            cv2.imwrite(filename, imgOriginal)
            cpt2Dollar=0
            cpt1Dollar=0
            cpt25Cents=0
            cpt10Cents=0
            cpt5Cents=0
            
            for pred in predictions['predictions']:
                # ajout de la valeur de chaque pièce à la somme
                
                if pred['class'] == '2Dollar':
                    sum += 2.0
                    cpt2Dollar+=1
                    
                elif pred['class'] == '1Dollar':
                    sum += 1.0
                    cpt1Dollar+=1

                    
                elif pred['class'] == '25Cents':
                    sum += 0.25
                    cpt25Cents+=1
                    
                elif pred['class'] == '10Cents':
                    sum += 0.10
                    cpt10Cents+=1
                    
                elif pred['class'] == '5Cents':
                    sum += 0.05
                    cpt5Cents+=1



            # afficher l'image avec contours dans le label
            imgAvecContours = Image.open("prediction.jpg")
            width, height = imgAvecContours.size
            imgAvecContours =  imgAvecContours.resize((450, 450))
            photo = ImageTk.PhotoImage(imgAvecContours)  
            labelImg.configure(image = photo)
            labelImg.image = photo
            label.config(text="Nous disposons de : \n\n "+str(cpt2Dollar)+ " || 2  Dollar   \n "+str(cpt1Dollar)+ " || 1  Dollar   \n "+str(cpt25Cents)+ " || 25  Cents \n "+str(cpt10Cents)+ " || 10  Cents \n "+str(cpt5Cents)+ " || 05 Cents \n\n La somme Total de piece est de : "+str(sum)+ "$")
    


In [107]:

#creation d'une boite 
frame = tk.Frame(window,bg='#14213D',bd=0,relief=tk.SUNKEN)

frame_boutons = tk.Frame(frame, bg='#14213D')
self_bouton = tk.Button(frame_boutons, text='Ouvrir une image  ', command=getImg, bg='#FCA311',fg='#FFFFFF',height=2)
self_bouton.pack(expand=True)
frame_boutons.pack(side="top", fill=tk.X, padx=10, pady=10,anchor=tk.CENTER)


frame_Img = tk.Frame(frame, bg='#14213D')
    # creer un label pour afficher une image
labelImg1 = tk.Label(frame_Img, bg='white', fg='white', )
imgAvecCont0 = Image.open("Imagenot.jpg")
width, height = imgAvecCont0.size
imgAvecCont0 =  imgAvecCont0.resize((450, 450))
photo0 = ImageTk.PhotoImage(imgAvecCont0)  
labelImg1.configure(image = photo0)
labelImg1.image = photo0
labelImg1.pack( side='left',expand=True)
    
# creer un label pour afficher une image
labelImg = tk.Label(frame_Img, bg='#14213D', fg='#14213D', )
   # afficher l'image avec contours dans le label
imgAvecCont = Image.open("Imagenot.jpg")
width, height = imgAvecCont.size
imgAvecCont =  imgAvecCont.resize((450, 450))
photo = ImageTk.PhotoImage(imgAvecCont)  
labelImg.configure(image = photo)
labelImg.image = photo
labelImg.pack(expand=True)
frame_Img.pack(side="top", fill=tk.X, padx=10, pady=10,anchor=tk.CENTER)

# Ajout d'un label pour afficher le texte
label = tk.Label(frame, text="Nous disposons de : \n\n **** 2    Dollar   \n **** 1    Dollar   \n **** 25  Cents \n **** 10  Cents \n ****  05 cents \n\n La somme Total de piece est de : ****Dollar", wraplength=400, bg='#14213D', fg='#FFFFFF',bd=1,relief=tk.SUNKEN)
label.pack(expand=True)

# Copyrigth
copyright = tk.Label(frame, text="\nCopyright \u00A9 Jonathan Kanyinda || Winner Pindi || Isaac Kampanga|| Kevin Nimi", bg='#14213D', fg='#FFFFFF')
copyright.pack(side="top", fill=tk.X, padx=10, pady=10,anchor=tk.CENTER)

In [ ]:

frame.pack()
#afficher fenetre
window.mainloop()